# NOTEBOOK PARA GENERAR EL DATAFRAME CON LA DATA DE TODOS LOS ARCHIVOS PARA AAFIT, BBFIT Y OTROS

<font size=5>La data para los hits la generaré con otro Notebook.</font>

Es importante mencionar que esta "limpieza" (generación del algoritmo) y obtención de data, me llevó aproximadamente 12 horas en general, desde el proceso creativo, pasando por la frustración, muchos intentos fallidos y hasta el momento de EUREKA!!!... XD.

Borré ya la mayoría de celdas que me llevaron al código de las siguientes dos celdas, el cual está mucho más pulido y limpio, pero creo que es importente mencionar por lo que pase y las ideas (algunas locas) que me surgieron para poder resolverlo.

Algunos ideas que tuve antes de llegar a decidir que no calcularia como tal el indice de aafit fueron:
* Solo quedarme con los archivos que tuvieran 100% data completa (terrible idea ya que son más del 50% los archivos que en algunos eventos - alrededor del 5% o menos - no tienen data de aafit).
* Verificar las longitudes de la cadena aafit y bbfit y comparar (idea bastante mala por cierto)
* Tratar de buscar por separado en otra celda solo la data de aafit y "pegarla" de alguna manera al dataframe final, hay que decir que esta idea también fué muy mala y no prosperó en lo absoluto (creo no tenia ni pies ni cabeza, momento de frustración máxima).

**NOTA:** Como siempre, la gran cuestión y prerrogativa humana es encontrar patrones en los datos y automatizar
búsquedas aún y cuando estos no se vean a simple vista o no sean fácilmente inferibles, tales como cuando 
el patrón es precisamente eso... que NO HAY PATRÓN!!!, esto aplica para el no indice ind_aafit

In [1]:
#Recordando algunas variables importantes, funciones y librerías que se requerirán
import os
import pandas as pd
import re
import math

#extractpath = "U:\MASTER INT. ARTIFICIAL\TFM_DATA\TXTs" #Este path es para mi laptop en casa
#extractpath = "F:\\DATA_TFM\\TXTs" #Este path es para mi PC en el trabajo
extractpath = "Z:\MASTER INT. ARTIFICIAL\TFM_DATA\TXTs" #Este es para mi no tan nueva PC-Desktop

txtfilesarr = os.listdir(extractpath); txtfilesarr=sorted(txtfilesarr)
#csv_path = "F:\\DATA_TFM\\CSVs\\"
csv_path = "Z:\MASTER INT. ARTIFICIAL\TFM_DATA\CSVs\\"
csvarr = os.listdir(csv_path);csvarr=sorted(csvarr)

# Funcion para obtener los indices de un archivo
def indices(df):
    ind_startev = [];ind_runid = [];ind_weights = [];ind_muon = []
    ind_bbfit = [];ind_bb_selp = [];ind_endev = [];#ind_aafit = []
    for i in range(data.shape[0]):
        tempstr = data[0][i]
        if "start_event" in tempstr: ind_startev.append(i)
        if "UTC" in tempstr: ind_runid.append(i)
        if "weights" in tempstr: ind_weights.append(i)
        if "muon" in tempstr: ind_muon.append(i)
        #if "aafit" in tempstr:  ##NO LO CALCULARE!!! OBTENDRE LOS VALORES DE OTRA FORMA
        #    ind_aafit.append(i) ##COMO TENGO EL INDICE DE BBFIT, RESTO 1 LINEA Y ASIGNO VALORES
        if "bbfit" in tempstr:   ##SEGUN LA EXISTENCIA O NO DE LA LINEA!!!! EUREKA!!!
            ind_bbfit.append(i)
        if "selected" in tempstr: ind_bb_selp.append(i)
        if "end_event" in tempstr: ind_endev.append(i)
    return(ind_startev,ind_runid,ind_weights,ind_muon,ind_bbfit,ind_bb_selp,ind_endev)

#Función para verificar si los indices tienen el mismo # de elementos
def longitudes(i1,i2,i3,i4,i5,i6,i7):
    a=len(i1);b=len(i2);c=len(i3);d=len(i4);e=len(i5);f=len(i6);g=len(i7)
    #print(a,b,c,d,e,f,g)
    if (a==b and b==c and c==d and d==e and e==f and f==g): return(True)
    else: return(False)
    return()

### NOTA: Como siempre, la gran cuestión y prerrogativa humana es encontrar patrones en los datos y automatizar
### búsquedas aún y cuando estos no se vean a simple vista o no sean fácilmente inferibles, tales como cuando 
### el patrón es precisamente eso... que NO HAY PATRÓN!!!, esto aplica para el no indice ind_aafit

## Se requiere poder generar un algoritmo que sea más rápido que el anterior, que metía los datos directo a un dataframe con loc... 

<font size=4>Ideas:</font>

1. Generar una lista en vez de un dataframe y al final generar el dataframe, a ver si resulta más rápido.

2. Sacar la data de cada archivo CSV y generar otro archivo ya con la data filtrada, así si se reinicia la PC, al menos no se pierde toda el trabajo, unicamente el del archivo último.

**NOTA:** Después de probar, la idea 1 funcionó muy bien, el tiempo de procesamiento de cada archivo, se redujo enormemente. Adicionalmente es muy importante mencionar que el uso de CPU de mi PC de casa no subió más allá del 50%, el anterior algoritmo que usaba el dataframe directo con loc, hacía que se fuera hasta el 100% de utilización.


In [2]:
############################################################################################################
###### ESTE CODIGO GENERARÁ UN DATAFRAME CON LA DATA DE BBFIT, AAFIT Y OTRAS DE ""TODOS"" LOS ARCHIVOS #####
############################################################################################################

#Se requiere un dataframe donde iremos guardando cada fila de data obtenida de cada archivo
columnas = ['runID','frameID','trigger_counter','interactionID','aafit_azimut',
            'aafit_zenit','aafit_lambda','aafit_beta','bbfit_azimut','bbfit_zenit','bbfit_quality',
            'muon_azimut','muon_zenit','muon_energy'] #La data del muon la añadi el 30 Jun, lo habia "olvidado" XD
data_df = pd.DataFrame(columns=columnas)
#La lista donde guardaremos los renglones antes de pasarla a dataframe
data_list = []

#Bucle para recorrer todos los archivos.
for k in range(len(csvarr)):
    file = csv_path+csvarr[k]
    #print("El archivo trabajando es: ",file,"eventos:",len(ind_))
    data = pd.read_csv(file, header=None) # Dataframe del archivo en tratamiento
    ind_startev = [];ind_runid = [];ind_weights = [];ind_muon = []
    ind_bbfit = [];ind_bb_selp = [];ind_endev = []
    #Generacion de indices para cada archivo
    ind_startev,ind_runid,ind_weights,ind_muon,ind_bbfit,ind_bb_selp,ind_endev = indices(data)
    print("El archivo trabajando es: ",file,"eventos:",len(ind_startev))
    #Como el runid es identico para todos los datos en un mismo archivo, uso el primero que aparece en el dataframe
    runID = int(list(str(data[0][ind_runid[0]]).split(" "))[0])
    #Igual como el interactionID es el mismo para todos, asigno 1 si el archivo es "numu" y 2 si es "anumu"
    interactionID = [lambda:1, lambda:2]["anumu" in str(file)]()
    #Bucle para recorrer el archivo en curso y obtener la data relevante
    print("Entrando al bucle para meter renglones a la lista") 
    for i in range(len(ind_startev)):
        #Codigo para obtener los demas datos por cada evento y despues asignarlo a los hits
        #Divido la cadena en los valores que la componen para después extraer frameid y trigger_counter
        temprunid = list(data[0][ind_runid[i]].split(" "))
        tempmuon = list(data[0][ind_muon[i]].split(" "))
        tempbbfit = list(data[0][ind_bbfit[i]].split(" "))
        #Extrayendo las variables frameid, trigger_counter de este renglon---> esta data si cambia ojo!!
        frameID = temprunid[1]; trigger_counter = temprunid [2]
        #Obteniendo y generando Datos para aafit
        #Para calcular la data solo haremos la diferencia y si exista data la calculamos si no
        #pues ponemos valores fuera de rango para saber después que no hubo data como -3pi
        laafit = data[0][ind_bbfit[i]-1]
        if "aafit" in laafit:
            tempaafit = list(data[0][ind_bbfit[i]-1].split(" "))
            aafit_azimut = math.atan2(float(tempaafit[3]),float(tempaafit[2]))
            aafit_zenit = math.acos(float(tempaafit[4]))
            aafit_lambda = tempaafit[-2]
            aafit_beta = tempaafit [-1]
        else:
            aafit_azimut=-3*math.pi;aafit_zenit = -3*math.pi;aafit_lambda=1;aafit_beta=-1
        #Obteniendo y generando Datos para bbfit
        X = float(tempbbfit[2]); Y = float(tempbbfit[3]); Z = float(tempbbfit[4])
        #Hay que verificar si X o Y son nan y asignar el valor de bbfit_azimut
        if math.isnan(X):   bbfit_azimut = -3*math.pi
        elif math.isnan(Y): bbfit_azimut = -3*math.pi
        else: bbfit_azimut = math.atan2(Y,X)
        bbfit_zenit = math.acos(Z)
        bbfit_quality = float(tempbbfit[-1]) 
        #Obteniendo y generando la data del muon
        X = float(tempmuon[1]); Y = float(tempmuon[2]); Z = float(tempmuon[3])
        muon_azimut = math.atan2(Y,X)
        muon_zenit = math.acos(Z)
        muon_energy = float(tempmuon[-2])
        #Añadimos la data obtenida y la generada a la lista "renglon"
        renglon = [runID, frameID, trigger_counter, interactionID,aafit_azimut, aafit_zenit,
                   aafit_lambda,aafit_beta, bbfit_azimut,bbfit_zenit,bbfit_quality,muon_azimut,muon_zenit,muon_energy]
        #Añadiendo los datos del "renglon" completo a la lista
        data_list.extend([renglon])

data_df = pd.DataFrame(data_list,columns=columnas)
print("Fin de la extracción de la data")

El archivo trabajando es:  Z:\MASTER INT. ARTIFICIAL\TFM_DATA\CSVs\MC_025800_anumu_CC_a_reco.i3.csv eventos: 256
Entrando al bucle para meter renglones a la lista
El archivo trabajando es:  Z:\MASTER INT. ARTIFICIAL\TFM_DATA\CSVs\MC_025800_numu_CC_a_reco.i3.csv eventos: 223
Entrando al bucle para meter renglones a la lista
El archivo trabajando es:  Z:\MASTER INT. ARTIFICIAL\TFM_DATA\CSVs\MC_025880_anumu_CC_a_reco.i3.csv eventos: 271
Entrando al bucle para meter renglones a la lista
El archivo trabajando es:  Z:\MASTER INT. ARTIFICIAL\TFM_DATA\CSVs\MC_025880_numu_CC_a_reco.i3.csv eventos: 230
Entrando al bucle para meter renglones a la lista
El archivo trabajando es:  Z:\MASTER INT. ARTIFICIAL\TFM_DATA\CSVs\MC_025920_anumu_CC_a_reco.i3.csv eventos: 273
Entrando al bucle para meter renglones a la lista
El archivo trabajando es:  Z:\MASTER INT. ARTIFICIAL\TFM_DATA\CSVs\MC_025920_numu_CC_a_reco.i3.csv eventos: 284
Entrando al bucle para meter renglones a la lista
El archivo trabajando es:

El archivo trabajando es:  Z:\MASTER INT. ARTIFICIAL\TFM_DATA\CSVs\MC_026870_numu_CC_a_reco.i3.csv eventos: 263
Entrando al bucle para meter renglones a la lista
El archivo trabajando es:  Z:\MASTER INT. ARTIFICIAL\TFM_DATA\CSVs\MC_026920_anumu_CC_a_reco.i3.csv eventos: 248
Entrando al bucle para meter renglones a la lista
El archivo trabajando es:  Z:\MASTER INT. ARTIFICIAL\TFM_DATA\CSVs\MC_026920_numu_CC_a_reco.i3.csv eventos: 213
Entrando al bucle para meter renglones a la lista
El archivo trabajando es:  Z:\MASTER INT. ARTIFICIAL\TFM_DATA\CSVs\MC_026940_anumu_CC_a_reco.i3.csv eventos: 320
Entrando al bucle para meter renglones a la lista
El archivo trabajando es:  Z:\MASTER INT. ARTIFICIAL\TFM_DATA\CSVs\MC_026940_numu_CC_a_reco.i3.csv eventos: 307
Entrando al bucle para meter renglones a la lista
El archivo trabajando es:  Z:\MASTER INT. ARTIFICIAL\TFM_DATA\CSVs\MC_026970_anumu_CC_a_reco.i3.csv eventos: 305
Entrando al bucle para meter renglones a la lista
El archivo trabajando es:

El archivo trabajando es:  Z:\MASTER INT. ARTIFICIAL\TFM_DATA\CSVs\MC_027900_anumu_CC_a_reco.i3.csv eventos: 249
Entrando al bucle para meter renglones a la lista
El archivo trabajando es:  Z:\MASTER INT. ARTIFICIAL\TFM_DATA\CSVs\MC_027900_numu_CC_a_reco.i3.csv eventos: 262
Entrando al bucle para meter renglones a la lista
El archivo trabajando es:  Z:\MASTER INT. ARTIFICIAL\TFM_DATA\CSVs\MC_027980_anumu_CC_a_reco.i3.csv eventos: 365
Entrando al bucle para meter renglones a la lista
El archivo trabajando es:  Z:\MASTER INT. ARTIFICIAL\TFM_DATA\CSVs\MC_027980_numu_CC_a_reco.i3.csv eventos: 392
Entrando al bucle para meter renglones a la lista
El archivo trabajando es:  Z:\MASTER INT. ARTIFICIAL\TFM_DATA\CSVs\MC_027990_anumu_CC_a_reco.i3.csv eventos: 369
Entrando al bucle para meter renglones a la lista
El archivo trabajando es:  Z:\MASTER INT. ARTIFICIAL\TFM_DATA\CSVs\MC_027990_numu_CC_a_reco.i3.csv eventos: 321
Entrando al bucle para meter renglones a la lista
El archivo trabajando es:

El archivo trabajando es:  Z:\MASTER INT. ARTIFICIAL\TFM_DATA\CSVs\MC_028980_numu_CC_a_reco.i3.csv eventos: 350
Entrando al bucle para meter renglones a la lista
El archivo trabajando es:  Z:\MASTER INT. ARTIFICIAL\TFM_DATA\CSVs\MC_028990_anumu_CC_a_reco.i3.csv eventos: 355
Entrando al bucle para meter renglones a la lista
El archivo trabajando es:  Z:\MASTER INT. ARTIFICIAL\TFM_DATA\CSVs\MC_028990_numu_CC_a_reco.i3.csv eventos: 316
Entrando al bucle para meter renglones a la lista
El archivo trabajando es:  Z:\MASTER INT. ARTIFICIAL\TFM_DATA\CSVs\MC_029040_anumu_CC_a_reco.i3.csv eventos: 308
Entrando al bucle para meter renglones a la lista
El archivo trabajando es:  Z:\MASTER INT. ARTIFICIAL\TFM_DATA\CSVs\MC_029040_numu_CC_a_reco.i3.csv eventos: 284
Entrando al bucle para meter renglones a la lista
El archivo trabajando es:  Z:\MASTER INT. ARTIFICIAL\TFM_DATA\CSVs\MC_029050_anumu_CC_a_reco.i3.csv eventos: 409
Entrando al bucle para meter renglones a la lista
El archivo trabajando es:

El archivo trabajando es:  Z:\MASTER INT. ARTIFICIAL\TFM_DATA\CSVs\MC_029580_anumu_CC_a_reco.i3.csv eventos: 284
Entrando al bucle para meter renglones a la lista
El archivo trabajando es:  Z:\MASTER INT. ARTIFICIAL\TFM_DATA\CSVs\MC_029580_numu_CC_a_reco.i3.csv eventos: 230
Entrando al bucle para meter renglones a la lista
El archivo trabajando es:  Z:\MASTER INT. ARTIFICIAL\TFM_DATA\CSVs\MC_029600_anumu_CC_a_reco.i3.csv eventos: 279
Entrando al bucle para meter renglones a la lista
El archivo trabajando es:  Z:\MASTER INT. ARTIFICIAL\TFM_DATA\CSVs\MC_029600_numu_CC_a_reco.i3.csv eventos: 236
Entrando al bucle para meter renglones a la lista
El archivo trabajando es:  Z:\MASTER INT. ARTIFICIAL\TFM_DATA\CSVs\MC_029740_anumu_CC_a_reco.i3.csv eventos: 271
Entrando al bucle para meter renglones a la lista
El archivo trabajando es:  Z:\MASTER INT. ARTIFICIAL\TFM_DATA\CSVs\MC_029740_numu_CC_a_reco.i3.csv eventos: 265
Entrando al bucle para meter renglones a la lista
El archivo trabajando es:

...

El archivo trabajando es:  Z:\MASTER INT. ARTIFICIAL\TFM_DATA\CSVs\MC_052590_anumu_CC_a_reco.i3.csv eventos: 1003
Entrando al bucle para meter renglones a la lista
El archivo trabajando es:  Z:\MASTER INT. ARTIFICIAL\TFM_DATA\CSVs\MC_052590_numu_CC_a_reco.i3.csv eventos: 969
Entrando al bucle para meter renglones a la lista
El archivo trabajando es:  Z:\MASTER INT. ARTIFICIAL\TFM_DATA\CSVs\MC_052610_anumu_CC_a_reco.i3.csv eventos: 1045
Entrando al bucle para meter renglones a la lista
El archivo trabajando es:  Z:\MASTER INT. ARTIFICIAL\TFM_DATA\CSVs\MC_052610_numu_CC_a_reco.i3.csv eventos: 1018
Entrando al bucle para meter renglones a la lista
El archivo trabajando es:  Z:\MASTER INT. ARTIFICIAL\TFM_DATA\CSVs\MC_052640_anumu_CC_a_reco.i3.csv eventos: 977
Entrando al bucle para meter renglones a la lista
El archivo trabajando es:  Z:\MASTER INT. ARTIFICIAL\TFM_DATA\CSVs\MC_052640_numu_CC_a_reco.i3.csv eventos: 868
Entrando al bucle para meter renglones a la lista
El archivo trabajando 

El archivo trabajando es:  Z:\MASTER INT. ARTIFICIAL\TFM_DATA\CSVs\MC_053860_anumu_CC_a_reco.i3.csv eventos: 1956
Entrando al bucle para meter renglones a la lista
El archivo trabajando es:  Z:\MASTER INT. ARTIFICIAL\TFM_DATA\CSVs\MC_053860_numu_CC_a_reco.i3.csv eventos: 1784
Entrando al bucle para meter renglones a la lista
El archivo trabajando es:  Z:\MASTER INT. ARTIFICIAL\TFM_DATA\CSVs\MC_053900_anumu_CC_a_reco.i3.csv eventos: 1905
Entrando al bucle para meter renglones a la lista
El archivo trabajando es:  Z:\MASTER INT. ARTIFICIAL\TFM_DATA\CSVs\MC_053900_numu_CC_a_reco.i3.csv eventos: 1801
Entrando al bucle para meter renglones a la lista
El archivo trabajando es:  Z:\MASTER INT. ARTIFICIAL\TFM_DATA\CSVs\MC_053910_anumu_CC_a_reco.i3.csv eventos: 1505
Entrando al bucle para meter renglones a la lista
El archivo trabajando es:  Z:\MASTER INT. ARTIFICIAL\TFM_DATA\CSVs\MC_053910_numu_CC_a_reco.i3.csv eventos: 1449
Entrando al bucle para meter renglones a la lista
El archivo trabajan

El archivo trabajando es:  Z:\MASTER INT. ARTIFICIAL\TFM_DATA\CSVs\MC_055230_numu_CC_a_reco.i3.csv eventos: 1090
Entrando al bucle para meter renglones a la lista
El archivo trabajando es:  Z:\MASTER INT. ARTIFICIAL\TFM_DATA\CSVs\MC_055240_anumu_CC_a_reco.i3.csv eventos: 34
Entrando al bucle para meter renglones a la lista
El archivo trabajando es:  Z:\MASTER INT. ARTIFICIAL\TFM_DATA\CSVs\MC_055240_numu_CC_a_reco.i3.csv eventos: 33
Entrando al bucle para meter renglones a la lista
El archivo trabajando es:  Z:\MASTER INT. ARTIFICIAL\TFM_DATA\CSVs\MC_055330_anumu_CC_a_reco.i3.csv eventos: 786
Entrando al bucle para meter renglones a la lista
El archivo trabajando es:  Z:\MASTER INT. ARTIFICIAL\TFM_DATA\CSVs\MC_055330_numu_CC_a_reco.i3.csv eventos: 649
Entrando al bucle para meter renglones a la lista
El archivo trabajando es:  Z:\MASTER INT. ARTIFICIAL\TFM_DATA\CSVs\MC_055360_anumu_CC_a_reco.i3.csv eventos: 268
Entrando al bucle para meter renglones a la lista
El archivo trabajando es: 

El archivo trabajando es:  Z:\MASTER INT. ARTIFICIAL\TFM_DATA\CSVs\MC_057130_numu_CC_a_reco.i3.csv eventos: 672
Entrando al bucle para meter renglones a la lista
El archivo trabajando es:  Z:\MASTER INT. ARTIFICIAL\TFM_DATA\CSVs\MC_057240_numu_CC_a_reco.i3.csv eventos: 743
Entrando al bucle para meter renglones a la lista
El archivo trabajando es:  Z:\MASTER INT. ARTIFICIAL\TFM_DATA\CSVs\MC_057250_anumu_CC_a_reco.i3.csv eventos: 985
Entrando al bucle para meter renglones a la lista
El archivo trabajando es:  Z:\MASTER INT. ARTIFICIAL\TFM_DATA\CSVs\MC_057260_anumu_CC_a_reco.i3.csv eventos: 1050
Entrando al bucle para meter renglones a la lista
El archivo trabajando es:  Z:\MASTER INT. ARTIFICIAL\TFM_DATA\CSVs\MC_057260_numu_CC_a_reco.i3.csv eventos: 1015
Entrando al bucle para meter renglones a la lista
El archivo trabajando es:  Z:\MASTER INT. ARTIFICIAL\TFM_DATA\CSVs\MC_057370_anumu_CC_a_reco.i3.csv eventos: 1228
Entrando al bucle para meter renglones a la lista
El archivo trabajando 

El archivo trabajando es:  Z:\MASTER INT. ARTIFICIAL\TFM_DATA\CSVs\MC_058510_anumu_CC_a_reco.i3.csv eventos: 948
Entrando al bucle para meter renglones a la lista
El archivo trabajando es:  Z:\MASTER INT. ARTIFICIAL\TFM_DATA\CSVs\MC_058510_numu_CC_a_reco.i3.csv eventos: 970
Entrando al bucle para meter renglones a la lista
El archivo trabajando es:  Z:\MASTER INT. ARTIFICIAL\TFM_DATA\CSVs\MC_058540_anumu_CC_a_reco.i3.csv eventos: 1253
Entrando al bucle para meter renglones a la lista
El archivo trabajando es:  Z:\MASTER INT. ARTIFICIAL\TFM_DATA\CSVs\MC_058540_numu_CC_a_reco.i3.csv eventos: 1250
Entrando al bucle para meter renglones a la lista
El archivo trabajando es:  Z:\MASTER INT. ARTIFICIAL\TFM_DATA\CSVs\MC_058550_anumu_CC_a_reco.i3.csv eventos: 1472
Entrando al bucle para meter renglones a la lista
El archivo trabajando es:  Z:\MASTER INT. ARTIFICIAL\TFM_DATA\CSVs\MC_058560_anumu_CC_a_reco.i3.csv eventos: 1299
Entrando al bucle para meter renglones a la lista
El archivo trabajand

In [4]:
print(data_df.shape[0])
data_df.sample(10)

1021092


,runID,frameID,trigger_counter,interactionID,aafit_azimut,aafit_zenit,aafit_lambda,aafit_beta,bbfit_azimut,bbfit_zenit,bbfit_quality,muon_azimut,muon_zenit,muon_energy
1010499,58560,27028,1834,1,0.628684,1.966673,-5.43218625997,0.00961232852203,0.791417,1.956598,5.591500,0.645242,1.965992,16337.70000
864273,54740,4348,1922,2,2.531675,0.430251,-6.81586459849,0.0103359031197,-9.424778,0.663012,2.286900,3.099454,0.857576,46.25450
210239,34640,5604,1252,1,-1.230033,0.855468,-4.96529444621,0.00691384423233,-1.120634,0.915423,3.388764,-1.231596,0.863373,198.58000
886522,55020,19982,2030,1,2.899592,0.367717,-6.92335391621,0.00910323574625,-9.424778,1.684303,3.537956,-0.246576,1.905766,1.67972
690038,51710,2903,1384,1,-1.175720,0.630231,-6.69792543436,0.0371529536145,-9.424778,1.011595,2.518997,-1.710641,1.042106,370.16500
106327,31700,3645,21,2,2.562168,1.044096,-6.64587654433,0.0216043462662,-9.424778,0.667270,2.987966,-2.886021,0.688583,30.13940
534215,46500,12095,1463,2,0.468708,1.363939,-5.68214342732,0.0509787565364,0.396880,1.531517,3.075022,-0.296668,1.649453,144.44300
952391,57400,23116,956,2,-2.655224,1.588238,-6.7207501619,0.0556673401526,-1.524424,1.594477,5.158282,-0.898833,2.367010,7.55189
759423,53230,50307,2604,2,-9.424778,-9.424778,1,-1,-9.424778,0.774384,3.170686,-2.620460,1.169140,86.25550
600888,49420,109611,834,2,-1.388765,0.552878,-6.47635758938,0.016021166497,-9.424778,0.840145,2.403417,1.792524,1.149187,112.26700


In [3]:
## Una vez que tenemos el dataframe con toda la data relevante de aafit y bbfit
## debemos guardar esta data a disco

outputpath = "Z:\MASTER INT. ARTIFICIAL\TFM_DATA\CLEANDATA\\"
    
data_df.to_csv (outputpath+"data_aafit_bbfit_muon"+".csv", index = None)#, header=None)

#Nota: Empezó a las 2:15pm del 30 Jun para 1136 archivos pero trono por actualizaciones el 3 Jul a las 7:20 p.m.
#Volvi a correr el proceso con listas y tardo de las 8:30p.m. a las 11:45 p.m. del 3 de Jul, con listas es wow!!

## NOTA MUY IMPORTANTE

De aquí en adelante, ya no utilicé el código, ya que este fue mi primer intento y utiliza un dataframe para guardar la data, sin embargo es extremadamente lento, yo pienso que porque utilicé la función loc. Lo que es muy cierto es que reutilicé la mayor parte y solo cambie la parte del dataframe, haciendo la asignación hasta el final.

Dejé el código y las celdas sólo como evidencia e historial de lo que he venido haciendo.

Ver el siguiente artículo: https://towardsdatascience.com/different-ways-to-iterate-over-rows-in-a-pandas-dataframe-performance-comparison-dc0d5dcef8fe


In [ ]:
############################################################################################################
###### ESTE CODIGO GENERARÁ UN DATAFRAME CON LA DATA DE BBFIT, AAFIT Y OTRAS DE ""TODOS"" LOS ARCHIVOS #####
############################################################################################################

#Se requiere un dataframe donde iremos guardando cada fila de data obtenida de cada archivo
columnas = ['runID','frameID','trigger_counter','interactionID','aafit_azimut',
            'aafit_zenit','aafit_lambda','aafit_beta','bbfit_azimut','bbfit_zenit','bbfit_quality',
            'muon_azimut','muon_zenit','muon_energy'] #La data del muon la añadi el 30 Jun, lo habia "olvidado" XD
df_data1 = pd.DataFrame(columns=columnas)

consecutivo=0 #variable auxiliar para guardar el indice del ultimo elemento del archivo

# Bucle para recorrer TODOS los archivos y obtener la data y metadata de TODOS
for k in range(len(csvarr)):
    #print("Ciclo", k)
    file = csv_path+csvarr[k]
    print("El archivo trabajando es: ",file)
    data = pd.read_csv(file, header=None) # Dataframe del archivo en tratamiento
    ind_startev = [];ind_runid = [];ind_weights = [];ind_muon = []
    ind_bbfit = [];ind_bb_selp = [];ind_endev = []

    #Generacion de indices para cada archivo
    ind_startev,ind_runid,ind_weights,ind_muon,ind_bbfit,ind_bb_selp,ind_endev = indices(data)

    #Como el runid es identico para todos los datos en un mismo archivo, uso el primero que aparece en el dataframe
    runID = int(list(str(data[0][ind_runid[0]]).split(" "))[0])
    #Igual como el interactionID es el mismo para todos, asigno 1 si el archivo es "numu" y 2 si es "anumu"
    interactionID = [lambda:1, lambda:2]["anumu" in str(file)]()
    # Bucle para recorrer el archivo en curso y obtener la data relevante
    print("Entrando al bucle para meter renglones al dataframe") 
    for i in range(len(ind_startev)):
        #Codigo para obtener los demas datos por cada evento y despues asignarlo a los hits
        #Divido la cadena en los valores que la componen para después extraer frameid y trigger_counter
        temprunid = list(data[0][ind_runid[i]].split(" "))
        tempmuon = list(data[0][ind_muon[i]].split(" "))
        tempbbfit = list(data[0][ind_bbfit[i]].split(" "))
        #Extrayendo las variables frameid, trigger_counter de este renglon---> esta data si cambia ojo!!
        frameID = temprunid[1]; trigger_counter = temprunid [2]
        #Obteniendo y generando Datos para aafit
        #Para calcular la data solo haremos la diferencia y si exista data la calculamos si no
        #pues ponemos valores fuera de rango para saber después que no hubo data como -3pi
        laafit = data[0][ind_bbfit[i]-1]
        if "aafit" in laafit:
            tempaafit = list(data[0][ind_bbfit[i]-1].split(" "))
            aafit_azimut = math.atan2(float(tempaafit[3]),float(tempaafit[2]))
            aafit_zenit = math.acos(float(tempaafit[4]))
            aafit_lambda = tempaafit[-2]
            aafit_beta = tempaafit [-1]
        else:
            aafit_azimut=-3*math.pi;aafit_zenit = -3*math.pi;aafit_lambda=1;aafit_beta=-1
        #Obteniendo y generando Datos para bbfit
        X = float(tempbbfit[2]); Y = float(tempbbfit[3]); Z = float(tempbbfit[4])
        #Hay que verificar si X o Y son nan y asignar el valor de bbfit_azimut
        if math.isnan(X):   bbfit_azimut = -3*math.pi
        elif math.isnan(Y): bbfit_azimut = -3*math.pi
        else: bbfit_azimut = math.atan2(Y,X)
        bbfit_zenit = math.acos(Z)
        bbfit_quality = float(tempbbfit[-1]) 
        #Obteniendo y generando la data del muon
        X = float(tempmuon[1]); Y = float(tempmuon[2]); Z = float(tempmuon[3])
        muon_azimut = math.atan2(Y,X)
        muon_zenit = math.acos(Z)
        muon_energy = float(tempmuon[-2])
        #Añadimos la data obtenida y la generada al dataframe
        renglon = [runID, frameID, trigger_counter, interactionID,aafit_azimut, aafit_zenit,
                   aafit_lambda,aafit_beta, bbfit_azimut,bbfit_zenit,bbfit_quality,muon_azimut,muon_zenit,muon_energy]
        #Añadiendo datos al dataframe.
        df_data1.loc[i+consecutivo] = renglon
    consecutivo = df_data1.shape[0]

print("Fin de la extracción de la data")

In [ ]:
### PRUEBAS... BORRAR ESTA CELDA AL FINAL

#Viendo como quedó el dataframe
print("El número de filas del dataframe es:", df_data1.shape[0])

print("Una muestra de las filas y la data en ellas")

df_data1.sample(10)

#df_data1.iloc[0:250]

# EL SIGUIENTE CÓDIGO NO ES NECESARIO PERO LO DEJÉ COMO EVIDENCIA DE LO QUE HABÍA ESTADO HACIENDO INICIALMENTE

In [ ]:
##########################################################################################
####  ESTE CODIGO CONTIENE EL PRIMER ALGORITMO QUE GENERÉ PARA OBTENER LA DATA DE      ###
####  UN ARCHIVO CSV YA CON TRATAMIENTO "INICIAL", ES DECIR SIN LINEAS VACIAS Y OTRAS  ### 
####          ESTABA DIVIDIDO EN DOS CELDAS POR TODO EL TESTING QUE HICE               ###
##########################################################################################

    ####  #####  #####  #####  #####
    ##    ####   ####   #   #  ####
    ####  #   #  #   #  #####  #   #
    
# -->ESTE ALGORITMO TUVO UN FALLO FUNDAMENTAL... NO TOMABA EN CUENTA QUE HAY ARCHIVOS QUE NO TIENEN DATA DE AAFIT
# -->Sin embargo me sirvió como base primaria para generar el adecuado (2da. celda de arriba a abajo de este notebook)

file = csv_path+csvarr[1]
print("El archivo trabajando es: ",file)
data = pd.read_csv(file, header=None) # Dataframe del archivo en tratamiento
ind_startev = [];ind_runid = [];ind_weights = [];ind_muon = []
ind_aafit = [];ind_bbfit = [];ind_bb_selp = [];ind_endev = []

for i in range(data.shape[0]):
    tempstr = data[0][i]
    if "start_event" in tempstr:
        ind_startev.append(i)
    if "UTC" in tempstr:
        ind_runid.append(i)
    if "weights" in tempstr:
        ind_weights.append(i)
    if "muon" in tempstr:
        ind_muon.append(i)
    if "aafit" in tempstr:
        ind_aafit.append(i)
    if "bbfit" in tempstr:
        ind_bbfit.append(i)
    if "selected" in tempstr:
        ind_bb_selp.append(i)
    if "end_event" in tempstr:
        ind_endev.append(i)

####### YA QUE LOGRE GENERAR EL DATAFRAME PARA LOS DATOS DE AAFIT Y BBBFIT DE UN SOLO EVENTO,
####### AHORA LO HARÉ PARA TODOS LOS EVENTOS EN EL MISMO ARCHIVO
####### CODIGO PARA OBTENER LA DATA AAFIT, BBFIT, TRANSFORMACIONES Y OTROS PARA EL DATAFRAME

#Como el runid es identico para todos los datos en un mismo archivo, uso el primero que aparece en el dataframe
runID = int(list(str(data[0][ind_runid[0]]).split(" "))[0])

#Igual como el interactionID es el mismo para todos, asigno 1 si el archivo es "numu" y 2 si es "anumu"
if "anumu" in str(file):
    interactionID = 2
else:
    interactionID = 1
    
#Se requiere un dataframe donde iremos guardando cada fila de data obtenida
columnas = ['runID','frameID','trigger_counter','interactionID','aafit_azimut','aafit_zenit',
            'aafit_lambda','aafit_beta','bbfit_azimut','bbfit_zenit','bbfit_quality']
df_data1 = pd.DataFrame(columns=columnas)

#Ahora generamos un bucle para recorrer todo el archivo y usamos los indices
for i in range(len(ind_startev)):
    #Codigo para obtener los demas datos por cada evento y despues asignarlo a los hits
    #Divido la cadena en los valores que la componen para después extraer frameid y trigger_counter
    temprunid = list(data[0][ind_runid[i]].split(" "))
    tempmuon = list(data[0][ind_muon[i]].split(" "))
    tempaafit = list(data[0][ind_aafit[i]].split(" "))
    tempbbfit = list(data[0][ind_bbfit[i]].split(" "))
    #Extrayendo las variables frameid, trigger_counter de este renglon---> esta data si cambia ojo!!
    frameID = temprunid[1]; trigger_counter = temprunid [2]
    #Obteniendo y generando Datos para aafit
    aafit_azimut = math.atan2(float(tempaafit[3]),float(tempaafit[2]))
    aafit_zenit = math.acos(float(tempaafit[4]))
    aafit_lambda = tempaafit[-2]
    aafit_beta = tempaafit [-1]
    #Obteniendo y generando Datos para bbfit
    X = float(tempbbfit[2]); Y = float(tempbbfit[3]); Z = float(tempbbfit[4])
    #Hay que verificar si X o Y son nan y asignar el valor de bbfit_azimut
    if math.isnan(X):   bbfit_azimut = -3*math.pi
    elif math.isnan(Y): bbfit_azimut = -3*math.pi
    else: bbfit_azimut = math.atan2(Y,X)
    bbfit_zenit = math.acos(Z)
    bbfit_quality = float(tempbbfit[-1]) 
    #Añadimos la data obtenida y la generada al dataframe
    renglon = [runID, frameID, trigger_counter, interactionID,aafit_azimut, aafit_zenit,
               aafit_lambda,aafit_beta, bbfit_azimut,bbfit_zenit,bbfit_quality]
    df_data1.loc[i] = renglon